# Getting Started - Defining your Experimental Setup

This guide shows you how to create the objects describing the experimental setup, which are needed for running the experiment workflows defined in the Applications Library.

## Reuse an existing DeviceSetup

If you already have a `DeviceSetup` for your experimental setup, that's great! You can use that. The Applications Library works on any setup, as long as the qubit UIDs match the names of the logical signal groups defined in the `DeviceSetup`. See [Define qubits](#Define-qubits) below.

## Create a DeviceSetup

You don't have `DeviceSetup`, start by creating one for your experimental setup. 

You have several options for defining your `DeviceSetup`: 

- Define your descriptor and setup by hand following the [Device Setup and Descriptor tutorial](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/tutorials/00_device_setup.html) in the LabOne Q Core documentation.
- Use the helper function `generate_descriptor` (comes with the `laboneq` package)
- Use the helper function `generate_device_setup` (comes with the `laboneq` package)
- Use the helper function `tunable_transmon_setup` (comes with the `laboneq-applications` package)

These three helper functions are targeted for an experimental setup containing qubits. The functions `generate_descriptor` and `generate_device_setup` can be used to generate a `DeviceSetup` with logical signal groups for `n` number of qubits and the instrument serial numbers that you have in your rack. `tunable_transmon_setup` is a convenient function to get a dummy, non-configurable `DeviceSetup` for `n` tunable-transmon qubits meant to be used in [emulation mode](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/01_session/concepts/00_session.html#emulation-mode) for quick prototyping.

Below, we show how to use these helper functions to create a `DeviceSetup` containing an SHFQC+ instrument, an HDAWG instrument, and a PQSC instrument, which are used to operate 3 qubits, labelled `q0, q1, q2`.

### generate_descriptor

The advantage of `generate_descriptor` is that it, by setting `get_zsync=True`, it automatically detects the zsync ports of the PQCS to which the other instruments in this descriptor are connected, and it queries the instruments for the [options]((https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/02_instrument_options.html)) that have been installed on them. Hence, you do not need to supply either the zsync connections or the instrument options by hand. You only need to specify the instrument serial numbers (DEVxxxx). 

In [ ]:
# Setting get_zsync=True automatically detects the zsync ports of the PQCS that
# are used by the other instruments in this descriptor.
# Here, we are not connected to instruments, so we set this flag to False.

from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor
from laboneq.simple import DeviceSetup

descriptor = generate_descriptor(
    pqsc=["DEV10001"],
    hdawg_8=["DEV8001"],
    shfqc_6=["DEV12001"],
    number_data_qubits=3,
    number_flux_lines=3,
    include_cr_lines=False,
    multiplex=True,
    number_multiplex=3,
    get_zsync=False,
    ip_address="localhost",
)
setup = DeviceSetup.from_descriptor(descriptor, "localhost")

The `setup` contains a logical signal group for each qubit labelled `q0, q1, q2`, and each of these qubit signal-line group contains the following signal lines: `drive_line`, `drive_line_ef`, `measure_line`, `acquire_line`, `flux_line`, as shown below.

In [ ]:
qubit_signals = {
    quid: list(lsg.logical_signals) for quid, lsg in setup.logical_signal_groups.items()
}
qubit_signals

### generate_device_setup

The advantage of `generate_device_setup` is that you have full control over how to configure your instruments in the `DeviceSetup`, by specifying any additional properties or [options of the instruments](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/02_instrument_options.html). Doing this is useful during development in emulation mode when you want to ensure that the emulated compilation is compatible with the options you have on your instruments in the lab.

In [ ]:
from laboneq.contrib.example_helpers.generate_device_setup import (
    generate_device_setup,
)

# specify the number of qubits you want to use
number_of_qubits = 3

# generate the device setup using a helper function
setup = generate_device_setup(
    number_qubits=number_of_qubits,
    pqsc=[{"serial": "DEV10001"}],
    hdawg=[
        {
            "serial": "DEV8001",
            "zsync": 0,
            "number_of_channels": 8,
            "options": "HDAWG8/CNT/MF/ME",
        }
    ],
    shfqc=[
        {
            "serial": "DEV12001",
            "zsync": 1,
            "number_of_channels": 6,
            "readout_multiplex": 3,
            "options": "SHFQC/QC6CH/PLUS/RTR",
        }
    ],
    include_flux_lines=True,
    multiplex_drive_lines=True,  # adds drive_ef
    server_host="localhost",
    setup_name="my_setup",
)

The `setup` contains a logical signal group for each qubit labelled `q0, q1, q2`, and each of these qubit signal-line group contains the following signal lines: `drive_line`, `drive_line_ef`, `measure_line`, `acquire_line`, `flux_line`, as shown below.

In [ ]:
qubit_signals = {
    quid: list(lsg.logical_signals) for quid, lsg in setup.logical_signal_groups.items()
}
qubit_signals

### tunable_transmon_setup

When you want to quickly prototype new experiments in [emulation mode](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/01_session/concepts/00_session.html#emulation-mode) and don't care about the exact details of the `DeviceSetup`, you can use the helper function `tunable_transmon_setup`. 

This function creates a dummy `DeviceSetup` containing an SHFQC+ instrument, an HDAWG instrument, and a PQSC instrument, which are used to operate `n_qubits` [tunable transmon qubits](https://docs.zhinst.com/labone_q_user_manual/applications_library/reference/qpu_types/tunable_transmon.html#laboneq_applications.qpu_types.tunable_transmon.TunableTransmonQubit), labelled `q0, q1, q2, ...` 

Let's use this function for `n_qubits=3`.

In [ ]:
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import (
    tunable_transmon_setup,
)

setup = tunable_transmon_setup(n_qubits=3)

The `setup` contains a logical signal group for each qubit labelled with the qubit UID, and each of these qubit signal-line group contains the following signal lines: `drive`, `drive_ef`, `measure`, `acquire`, `flux`, as shown below.

In [ ]:
qubit_signals = {
    quid: list(lsg.logical_signals) for quid, lsg in setup.logical_signal_groups.items()
}
qubit_signals

### Inspect the qubit-instrument connectivity

Use either of the three `DeviceSetups` defined above to inspect the connectivity between the instruments and the lines of the qubits:

In [ ]:
def get_physical_signal_name(quid, signal_name):
    logical_signal = setup.logical_signal_groups[quid].logical_signals[signal_name]
    return logical_signal.physical_channel.uid


qubit_signals = {
    quid: list(lsg.logical_signals) for quid, lsg in setup.logical_signal_groups.items()
}
connections = {
    quid: {sig_name: get_physical_signal_name(quid, sig_name) for sig_name in signals}
    for quid, signals in qubit_signals.items()
}

from pprint import pprint

pprint(connections)  # noqa: T203

We see that the three qubits are read out in parallel on the same quantum analyzer (QA) channel of the SHFQC instrument, and that their drive lines are controlled from individual signal generation (SG) channels of the SHFQC instrument. Finally, the flux lines of the qubits are controlled by individual HDAWG outputs. 

## Define qubits

We will show how to create qubit instances from either of the three `DeviceSetups` defined above. Here, we use the `TunableTransmonQubit` class with corresponding `TunableTransmonQubitParameters`, but the procedure is the same for any other child class of the LabOne Q [QuantumElements class](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/03_sections_pulses_and_quantum_operations/tutorials/04_quantum_elements.html).

<div class="alert alert-block alert-info">
<b>IMPORTANT</b><br> The names of the logical signals in your `DeviceSetup` must match the signal names expected by the qubit class. You can check the latter via QubitClassName.REQUIRED_SIGNALS and QubitClassName.OPTIONAL_SIGNALS.
</div>

In [ ]:
from laboneq_applications.qpu_types.tunable_transmon import (
    TunableTransmonQubit,
)

qubits = TunableTransmonQubit.from_device_setup(setup)

By using `TunableTransmonQubit.from_device_setup`, the qubits are instantiated with the logical signals given by the signals contained in the logical signal group in the `DeviceSetup` that has the same name as the qubit UID. So for example, if the logical signal groups in your `DeviceSetup` contain the logical signals "drive", "measure", "acquire", then the qubits will also have these signals. 

Check that the qubits we've created have the signals you expect from the `DeviceSetup`:

In [ ]:
for q in qubits:
    print(q.uid)
    for sig, lsg in q.signals.items():
        print(f"\t'{sig}:\t'{lsg}'")

The qubits are instantiated with identical default values of the `TunableTransmonQubitParameters` class. Let's see what the qubit parameters are:

In [ ]:
qubits[0].parameters

Check out the [tutorial on qubits and quantum operations](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/quantum_operations.html) to learn more about these parameters and how they are used in the Applications Library together with quantum operations. 

Adjust the values of the qubit parameters to the ones for your quantum device. You can change the value of any of the parameters as shown below for the `drive_lo_frequency` parameter:

In [ ]:
qubits[0].parameters.drive_lo_frequency = 6e9

If you already have the correct qubit parameters stored in an instance of `TunableTransmonQubitParameters` (for example, loaded from a file), you can directly pass them to the `parameters` argument of `TunableTransmonQubit.from_logical_signal_group`, and the qubits will be created with those parameters. 

## Define quantum operations

Next, we need to define the class of quantum operations implementing gates and operations on the qubits defined above. Here, we will use an instance of `TunableTransmonOperations` for the tunable transmons defined above.

In [ ]:
from laboneq_applications.qpu_types.tunable_transmon import TunableTransmonOperations

quantum_operations = TunableTransmonOperations()
quantum_operations.keys()

To learn more about quantum operations and how they are used to create quantum experiments from the qubit parameters, see [the tutorial on qubits and quantum operations](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/quantum_operations.html).

## Define the QPU

Finally, we define the quantum processor (QPU) containing the qubits and the corresponding quantum operations.

The `qpu` contains the source of ground truth for an experiment and the best state of knowledge of the quantum system that is being operated. This means that the parameters of the qubits and any other parameters of the QPU define the configuration used by all the experiments in the Applications Library. 

In [ ]:
from laboneq.dsl.quantum import QPU

qpu = QPU(qubits, quantum_operations=quantum_operations)

## Loading From a File

The qubits and QPU can also be loaded back from `json` files saved by an experiment in the Applications Library. You just need the path to the file:

```python
from laboneq import serializers

serializers.load(path_to_file)
```

## Optional: define a QuantumPlatform

Optionally, you can collect the `setup` and the `qpu` in an instance of `QuantumPlatform`.

In [ ]:
from laboneq.dsl.quantum import QuantumPlatform

qt_platform = QuantumPlatform(setup=setup, qpu=qpu)

## Demo QuantumPlatform

The [tunable_transmon_setup](#tunable_transmon_setup) and a `QPU` for `n` tunable transmon qubits defined above can also be more conveniently obtained by instantiating a demo quantum platform provided by the Application Library. This demo platform is useful for quick prototyping in emulation mode. 

In [ ]:
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

demo_qt_platform = demo_platform(n_qubits=3)

In [ ]:
log_sig_groups = demo_qt_platform.setup.logical_signal_groups
qubit_signals = {
    quid: list(lsg.logical_signals) for quid, lsg in log_sig_groups.items()
}
qubit_signals

In [ ]:
demo_qt_platform.qpu.qubits[0].parameters

In [ ]:
demo_qt_platform.qpu.quantum_operations.keys()

## Connect to Session

Now let's connect to a LabOne Q `Session`. Here, we connect in emulation mode. When running on real hardware, connect using `do_emulation=False`.

In [ ]:
from laboneq.simple import Session

session = Session(setup)
session.connect(do_emulation=True)

Great! You have created everything you need to get started with the measurements. Now, on to experiments!